<a href="https://colab.research.google.com/github/MatteoGuglielmi-tech/Polarity-and-Subjectivity-Detection/blob/main/src/MyModel/pre-processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 17444211612808208740
 xla_global_id: -1]

In [2]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
address sizes

In [ ]:
!cat /proc/meminfo

MemTotal:       13297220 kB
MemFree:         7433532 kB
MemAvailable:   11637900 kB
Buffers:          330324 kB
Cached:          3919580 kB
SwapCached:            0 kB
Active:           710736 kB
Inactive:        4908880 kB
Active(anon):       1016 kB
Inactive(anon):  1236264 kB
Active(file):     709720 kB
Inactive(file):  3672616 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:              1332 kB
Writeback:             0 kB
AnonPages:       1369596 kB
Mapped:           533500 kB
Shmem:              1272 kB
KReclaimable:     127224 kB
Slab:             160124 kB
SReclaimable:     127224 kB
SUnreclaim:        32900 kB
KernelStack:        5904 kB
PageTables:        24716 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6648608 kB
Committed_AS:    4134604 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       10724 kB
VmallocChunk:          0 kB
Percpu:          

# MTL and BERT Embedding

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
rootdir = '/content/gdrive/MyDrive/Colab Notebooks/Polarity-Subjectivity-Detection/'

In [5]:
from nltk.sentiment.util import mark_negation
from typing import List, Dict, Tuple

def negative_marking(doc : List[str]) -> List[str]:
    '''
        Params :
        -----------------
            doc : list[str]
                document where each element is a list of strings
        Returns :
            negated_doc : list[str]
                document after having applied double negation
    '''

    flat_doc = [w for sent in doc for w in sent]
    negated_doc = mark_negation(flat_doc, double_neg_flip=True)

    return " ".join([w for w in negated_doc])

In [16]:
from nltk.corpus import stopwords
import re
import string
from textblob import TextBlob as tb


## https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python
# since re are largely used for this type of applications, the regex module is used
# re.sub(pattern, repl, string, count=0, flags=0)

def pre_processing(text : str) -> str :
  '''Clear text from numbers, stop words (very common words), punctuation and 
    correct possible misspelled words

    Params :
    --------
      cw : List[str]
        list of words in the sentence to be cleaned
    Returns :
    ---------
      list of cleaned words
  '''

  stop_list = set(stopwords.words("english"))
  text = list(text.lower().split())
  text = ' '.join([word for word in text if word not in stop_list])
  # remove http links
  text = re.sub(r'http\S+', '', text)
  # Remove hashtags
  text = re.sub(r'#\w*', '', text)
  # Remove whitespace (including new line characters)
  text = re.sub(r'\s\s+', '', text)
  # Remove single space remaining at the front of the tweet.
  text = text.lstrip(' ') 
  # Remove @username
  text = re.sub('@[^\s]+','', text)
  text = list(text.translate(str.maketrans('', '', string.punctuation)).split())
  # correction of possible miss-click
  #text = ' '.join([str(tb(word).correct()) for word in text])
  text = ' '.join([str(word) for word in text])

  return text

In [18]:
import re
from string import punctuation

In [6]:
def process(tweet):
    tweet = re.sub(r'\n', '', tweet)
    # Remove HTML special entities
    tweet = re.sub(r'\&\w*;', '', tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','',tweet)
    # Remove tickers
    tweet = re.sub(r'\$\w*', '', tweet)
    # To lowercase
    tweet = tweet.lower()
    # Remove hyperlinks
    # tweet = re.sub(r'https?:\/\/.*\/\w*', '', tweet)
    tweet = re.sub(r"http\S+", "", tweet)
    # Remove hashtags
    tweet = re.sub(r'#\w*', '', tweet)
    # Remove Punctuation and split 's, 't, 've with a space for filter
    tweet = re.sub(r'[' + punctuation.replace('@', '') + ']+', ' ', tweet)
    # Remove words with 2 or fewer letters
    tweet = re.sub(r'\b\w{1,2}\b', '', tweet)
    # Remove whitespace (including new line characters)
    tweet = re.sub(r'\s\s+', ' ', tweet)
    # Remove single space remaining at the front of the tweet.
    tweet = tweet.lstrip(' ') 
    # Remove characters beyond Basic Multilingual Plane (BMP) of Unicode:
    tweet = ''.join(c for c in tweet if c <= '\uFFFF') 
    return tweet

In [7]:
import nltk

In [8]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('movie_reviews')
nltk.download('subjectivity')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
[nltk_data] Downloading package subjectivity to /root/nltk_data...
[nltk_data]   Unzipping corpora/subjectivity.zip.


True

In [76]:
from textblob.en import subjectivity
from nltk.corpus import movie_reviews
from nltk.corpus import subjectivity


mr = movie_reviews
sub = subjectivity
neg = mr.paras(categories='neg')
pos = mr.paras(categories='pos')


subj_docs = [sent for sent in subjectivity.sents(categories='subj')]
obj_docs = [sent for sent in subjectivity.sents(categories='obj')]

print(len(neg), len(pos))
print(len(subj_docs), len(obj_docs))

1000 1000
5000 5000


### Polarity Dataset Analysis

In [10]:
for sent in pos[0]:
    print(sent)

['films', 'adapted', 'from', 'comic', 'books', 'have', 'had', 'plenty', 'of', 'success', ',', 'whether', 'they', "'", 're', 'about', 'superheroes', '(', 'batman', ',', 'superman', ',', 'spawn', ')', ',', 'or', 'geared', 'toward', 'kids', '(', 'casper', ')', 'or', 'the', 'arthouse', 'crowd', '(', 'ghost', 'world', ')', ',', 'but', 'there', "'", 's', 'never', 'really', 'been', 'a', 'comic', 'book', 'like', 'from', 'hell', 'before', '.']
['for', 'starters', ',', 'it', 'was', 'created', 'by', 'alan', 'moore', '(', 'and', 'eddie', 'campbell', ')', ',', 'who', 'brought', 'the', 'medium', 'to', 'a', 'whole', 'new', 'level', 'in', 'the', 'mid', "'", '80s', 'with', 'a', '12', '-', 'part', 'series', 'called', 'the', 'watchmen', '.']
['to', 'say', 'moore', 'and', 'campbell', 'thoroughly', 'researched', 'the', 'subject', 'of', 'jack', 'the', 'ripper', 'would', 'be', 'like', 'saying', 'michael', 'jackson', 'is', 'starting', 'to', 'look', 'a', 'little', 'odd', '.']
['the', 'book', '(', 'or', '"', 'g

In [11]:
def lol2str(doc):
    # flatten & join
    return " ".join([w for sent in doc for w in sent])

In [49]:
#new_corpus = [negative_marking(d) for d in pos] + [negative_marking(d) for d in neg]
new_corpus = [lol2str(d) for d in pos] + [lol2str(d) for d in neg]

In [50]:
import pandas as pd

data = pd.DataFrame(new_corpus, columns=['text'])

In [51]:
data

,text
0,films adapted from comic books have had plenty...
1,every now and then a movie comes along from a ...
2,you ' ve got mail works alot better than it de...
3,""" jaws "" is a rare film that grabs your attent..."
4,moviemaking is a lot like being the general ma...
...,...
1995,"if anything , "" stigmata "" should be taken as ..."
1996,"john boorman ' s "" zardoz "" is a goofy cinemat..."
1997,the kids in the hall are an acquired taste . i...
1998,there was a time when john carpenter was a gre...


In [52]:
#pre_processing('it is a beautiful lif, https://github @matthew')
process('it is a beautiful lif, https://github @matthew')

'beautiful lif '

In [53]:
#data['text'] = data['text'].apply(pre_processing)
data['text'] = data['text'].apply(process)

In [54]:
data

,text
0,films adapted from comic books have had plenty...
1,every now and then movie comes along from susp...
2,you got mail works alot better than deserves o...
3,jaws rare film that grabs your attention befor...
4,moviemaking lot like being the general manager...
...,...
1995,anything stigmata should taken warning against...
1996,john boorman zardoz goofy cinematic debacle fu...
1997,the kids the hall are acquired taste took leas...
1998,there was time when john carpenter was great h...


In [55]:
labels = [[1,0]] * (len(data['text'])//2) + [[0,1]] * (len(data['text'])//2)
len(labels)

2000

In [56]:
labels = [1 if idx<(len(data['text'])//2) else 0 for idx in range(len(data['text']))]

In [57]:
len(labels[0:1000])

1000

In [58]:
len(labels[1000:])

1000

In [59]:
df = pd.DataFrame(labels, columns=['labels'])

In [60]:
complete_data = pd.concat([data, df], axis=1)

In [61]:
complete_data

,text,labels
0,films adapted from comic books have had plenty...,1
1,every now and then movie comes along from susp...,1
2,you got mail works alot better than deserves o...,1
3,jaws rare film that grabs your attention befor...,1
4,moviemaking lot like being the general manager...,1
...,...,...
1995,anything stigmata should taken warning against...,0
1996,john boorman zardoz goofy cinematic debacle fu...,0
1997,the kids the hall are acquired taste took leas...,0
1998,there was time when john carpenter was great h...,0


In [62]:
# saving built dataset
complete_data.to_csv(rootdir+'Datasets/movie_rews.csv')
data.to_csv(rootdir+'Datasets/movie_rews_clean.csv')
new_corp_df  = pd.DataFrame(new_corpus, columns=['text'])
new_corp_df.to_csv(rootdir+'Datasets/movie_rews_raw.csv')

This to show that $99\%$ of the sentences are under $947.14$ words.

In [63]:
import numpy as np
from nltk.tokenize import word_tokenize


seq_len = np.array([len(word_tokenize(sent)) for sent in complete_data['text']])
print([(p, np.percentile(seq_len, p)) for p in [65, 75, 80, 85, 90, 95, 99, 100]])

[(65, 569.0), (75, 633.25), (80, 671.0), (85, 721.1499999999999), (90, 804.1000000000001), (95, 951.1999999999998), (99, 1288.06), (100, 1882.0)]


In [64]:
avg_length = seq_len.mean()
avg_length

523.155

### Subjectivity Dataset Analysis

In [65]:
subj_corpus = [" ".join(sent) for sent in subj_docs] + [" ".join(sent) for sent in obj_docs]
subj_labels = np.array([1] * len(subj_docs) + [0] * len(obj_docs))

In [66]:
subj_corpus_df = pd.DataFrame(subj_corpus, columns=['text'])
subj_labels_df = pd.DataFrame(subj_labels, columns=['labels'])

subj_obj_dataset = pd.concat([subj_corpus_df, subj_labels_df], axis=1)
subj_obj_dataset['text'] = subj_obj_dataset['text'].apply(process) 

In [67]:
subj_corpus_df

,text
0,"smart and alert , thirteen conversations about..."
1,"color , musical bounce and warm seas lapping o..."
2,it is not a mass-market entertainment but an u...
3,a light-hearted french film about the spiritua...
4,my wife is an actress has its moments in looki...
...,...
9995,"in the end , they discover that balance in lif..."
9996,a counterfeit 1000 tomin bank note is passed i...
9997,enter the beautiful and mysterious secret agen...
9998,after listening to a missionary from china spe...


In [68]:
subj_obj_dataset

,text,labels
0,smart and alert thirteen conversations about o...,1
1,color musical bounce and warm seas lapping isl...,1
2,not mass market entertainment but uncompromisi...,1
3,light hearted french film about the spiritual ...,1
4,wife actress has its moments looking the comic...,1
...,...,...
9995,the end they discover that balance life simila...,0
9996,counterfeit 1000 tomin bank note passed bazaar,0
9997,enter the beautiful and mysterious secret agen...,0
9998,after listening missionary from china speak ch...,0


In [69]:
# saving built dataset
subj_obj_dataset.to_csv(rootdir+'Datasets/subj_obj_dataset.csv')
subj_corpus_df.to_csv(rootdir+'Datasets/subj_obj_dataset_clean.csv')

In [70]:
import numpy as np
from nltk.tokenize import word_tokenize


#seq_len = np.array([len(word_tokenize(sent)) for sent in subj_obj_dataset['text']])
seq_len = np.array([len(word_tokenize(sent)) for sent in subj_corpus_df['text']])
print([(p, np.percentile(seq_len, p)) for p in [65, 75, 80, 85, 90, 95, 99, 100]])

[(65, 27.0), (75, 30.0), (80, 32.0), (85, 35.0), (90, 38.0), (95, 43.0), (99, 56.0), (100, 122.0)]


In [72]:
avg_length = seq_len.mean()
avg_length

24.6031

[Bert Embedding](https://www.youtube.com/watch?v=zJW57aCBCTk)

[Fine-Tuning Bert](https://www.youtube.com/watch?v=x66kkDnbzi4)

### Unused

In [ ]:
def arrange_subjectivity(subjective_sents : List[Tuple[List[str], str]]) -> pd.DataFrame:
  '''Arrange subjectivity dataset into a table with format (index, text, label)
    Params:
    ------
      subjective_sents : list(tuple(list(str),str))
        subjectivity dataset
    Returns:
    ------
      A dataframe type resembling a table organized as follow : (insed, sentence text, label)
  '''

  sents = []
  labels = []
  counter =0
  for words, label in subjective_sents:
    sents.append(' '.join([w for w in words]))
    labels.append(label)
    
  df_sents = pd.DataFrame(sents, columns=['text'])
  df_labels = pd.DataFrame(labels, columns=["tag"])
  df = pd.concat([df_sents, df_labels], axis=1)

  return df

In [ ]:
docs = subj_docs + obj_docs
subj_obj_dataset = arrange_subjectivity(docs)
subj_obj_dataset

ValueError: ignored